In [43]:
from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy

#import spacy, and spacy french model
# spacy is used to work on text
import spacy
nlp = spacy.load('en')


In [11]:

#import other libraries
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle


In [26]:

#define parameters used in the tutorial
data_dir = '/media/dheeraj/9A26F0CB26F0AA01/WORK/IIITD/NLP/Allrecipes/Allrecipes/African/African Deltas'# data directory containing raw texts
save_dir = '/media/dheeraj/9A26F0CB26F0AA01/WORK/IIITD/NLP/Allrecipes/Allrecipes/African/African Deltas/save' # directory to store trained NN models


vocab_file = os.path.join(save_dir, "words_vocab.pkl")
sequences_step = 1 #step to create sequences

In [17]:
def create_wordlist(doc):
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n",'\u2009','\xa0'):
            wl.append(word.text.lower())
    return wl

In [56]:
wordlist = []

for file_name in os.listdir(data_dir):
    input_file = os.path.join(data_dir, file_name)
    #read data
    print(input_file)
    with codecs.open(input_file, "r") as f:
        data = f.read()
        
    #create sentences
    doc = nlp(data)
    wl = create_wordlist(doc)
    wordlist = wordlist + wl
    
    
print(wordlist)

/media/dheeraj/9A26F0CB26F0AA01/WORK/IIITD/NLP/Allrecipes/Allrecipes/African/African Deltas/Best Hot Sauce.txt
/media/dheeraj/9A26F0CB26F0AA01/WORK/IIITD/NLP/Allrecipes/Allrecipes/African/African Deltas/Chin Chin.txt
/media/dheeraj/9A26F0CB26F0AA01/WORK/IIITD/NLP/Allrecipes/Allrecipes/African/African Deltas/Guinean Okra Sauce.txt
/media/dheeraj/9A26F0CB26F0AA01/WORK/IIITD/NLP/Allrecipes/Allrecipes/African/African Deltas/Malian Ginger Juice.txt
/media/dheeraj/9A26F0CB26F0AA01/WORK/IIITD/NLP/Allrecipes/Allrecipes/African/African Deltas/save


IsADirectoryError: [Errno 21] Is a directory: '/media/dheeraj/9A26F0CB26F0AA01/WORK/IIITD/NLP/Allrecipes/Allrecipes/African/African Deltas/save'

In [27]:
# count the number of words
word_counts = collections.Counter(wordlist)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)

#save the words and vocabulary
with open(os.path.join(vocab_file), 'wb') as f:
    cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  480


In [32]:
#create sequences
sequences = []
next_words = []
seq_length=30
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 1664


In [53]:
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
print(X.shape)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1

(1664, 30, 480)


In [34]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    print("model built!")
    return model

In [35]:
rnn_size = 256 # size of RNN
seq_length = 30 # sequence length
learning_rate = 0.001 #learning rate

md = bidirectional_lstm_model(seq_length, vocab_size)
md.summary()

Build LSTM model.
model built!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               1509376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 480)               246240    
_________________________________________________________________
activation_1 (Activation)    (None, 480)               0         
Total params: 1,755,616
Trainable params: 1,755,616
Non-trainable params: 0
_________________________________________________________________


In [57]:
batch_size = 32 # minibatch size
num_epochs = 50 # number of epochs

checkpoints=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath=save_dir + "/" + 'my_model_gen_sentences.{epoch:02d}-{val_loss:.2f}.hdf5',\
                           monitor='val_loss', verbose=0, mode='auto', period=2)]
#fit the model
history = md.fit(X, y,
                 batch_size=batch_size,
                 shuffle=True,
                 epochs=num_epochs,
                 validation_split=0.1, 
                 callbacks=checkpoints)
                 
#save the model
md.save(save_dir + "/" + 'my_model_generate_sentences.h5')

Train on 1497 samples, validate on 167 samples
Epoch 1/50
1497/1497 [==============================] - 36s 24ms/step - loss: 4.2468 - categorical_accuracy: 0.1242 - val_loss: 6.0969 - val_categorical_accuracy: 0.1018
Epoch 2/50
1497/1497 [==============================] - 38s 25ms/step - loss: 4.1290 - categorical_accuracy: 0.1530 - val_loss: 6.0906 - val_categorical_accuracy: 0.0838
Epoch 3/50
1497/1497 [==============================] - 39s 26ms/step - loss: 3.9725 - categorical_accuracy: 0.1717 - val_loss: 6.1453 - val_categorical_accuracy: 0.0958
Epoch 4/50
1497/1497 [==============================] - 41s 28ms/step - loss: 3.7932 - categorical_accuracy: 0.1844 - val_loss: 5.9997 - val_categorical_accuracy: 0.1078
Epoch 5/50
1497/1497 [==============================] - 48s 32ms/step - loss: 3.6716 - categorical_accuracy: 0.2104 - val_loss: 6.4031 - val_categorical_accuracy: 0.0898
Epoch 6/50
1497/1497 [==============================] - 47s 32ms/step - loss: 3.4910 - categorical_accu